In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

In [17]:
base_path = "data/assignment"
orders = pd.read_csv(f"{base_path}/Train/orders.csv")
train_customers = pd.read_csv(f"{base_path}/Train/train_customers.csv")
vendors = pd.read_csv(f"{base_path}/Train/vendors.csv")
test_customers = pd.read_csv(f"{base_path}/Test/test_customers.csv")
test_locations = pd.read_csv(f"{base_path}/Test/test_locations.csv")
sample_submission = pd.read_csv(f"{base_path}/SampleSubmission.csv")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24812\3490293822.py:2: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv(f"{base_path}/Train/orders.csv")


In [18]:
print("orders columns:", orders.columns.tolist())
print("train_customers columns:", train_customers.columns.tolist())


orders columns: ['order_id', 'customer_id', 'item_count', 'grand_total', 'payment_mode', 'promo_code', 'vendor_discount_amount', 'promo_code_discount_percentage', 'is_favorite', 'is_rated', 'vendor_rating', 'driver_rating', 'deliverydistance', 'preparationtime', 'delivery_time', 'order_accepted_time', 'driver_accepted_time', 'ready_for_pickup_time', 'picked_up_time', 'delivered_time', 'delivery_date', 'vendor_id', 'created_at', 'LOCATION_NUMBER', 'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR']
train_customers columns: ['customer_id', 'gender', 'dob', 'status', 'verified', 'language', 'created_at', 'updated_at']


In [19]:
print(vendors.columns)


Index(['id', 'authentication_id', 'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_haked_delivering',
       'discount_percentage', 'status', 'verified', 'rank', 'language',
       'vendor_rating', 'sunday_from_time1', 'sunday_to_time1',
       'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
       'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
       'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2',
       'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1',
       'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1',
       'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
       'friday_from_time1', 'friday_to_time1', 'friday_from_time2',
       'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1',
       'saturday_from_time2', 'saturday

In [24]:
# 3. Merge for positive samples
orders["vendor_id"] = orders["vendor_id"].astype(str)
vendors["id"] = vendors["id"].astype(str)

df_pos = orders.merge(train_customers, on="customer_id", how="left")
df_pos = df_pos.merge(vendors, left_on="vendor_id", right_on="id", how="left")
df_pos["target"] = 1
df_pos = df_pos[["customer_id", "LOCATION_NUMBER", "vendor_id", "target"]]
df_pos.columns = ["CID", "LOC_NUM", "VENDOR", "target"]


In [ ]:
# 4. Create negative samples (Fixed)
neg_samples = []
for cid in unique_customers:
    pos_vendors = df_pos[df_pos["CID"] == cid]["VENDOR"].tolist()
    available_neg_vendors = [v for v in unique_vendors if v not in pos_vendors]
    
    if len(available_neg_vendors) == 0:
        continue  # skip this customer
    
    neg_size = min(5, len(available_neg_vendors))
    neg_vendors = np.random.choice(available_neg_vendors, size=neg_size, replace=False)
    
    for v in neg_vendors:
        neg_samples.append([cid, 0, v, 0])

df_neg = pd.DataFrame(neg_samples, columns=["CID", "LOC_NUM", "VENDOR", "target"])
df_all = pd.concat([df_pos, df_neg], ignore_index=True)


In [22]:
# 5. Encode categorical
df_all["CID"] = df_all["CID"].astype("category").cat.codes
df_all["VENDOR"] = df_all["VENDOR"].astype("category").cat.codes


NameError: name 'df_all' is not defined

In [ ]:
# 6. Split
X = df_all.drop("target", axis=1)
y = df_all["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X

NameError: name 'X' is not defined

In [ ]:
# 7. Model
model = LGBMClassifier()
model.fit(X_train, y_train)
preds = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, preds)
print("ROC AUC:", roc_auc)


[LightGBM] [Info] Number of positive: 108478, number of negative: 109703
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014827 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 218181, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497193 -> initscore=-0.011229
[LightGBM] [Info] Start training from score -0.011229
ROC AUC: 0.8547388857039672


In [ ]:
import os
os.makedirs("model", exist_ok=True)  # Creates the 'model' folder if it doesn't exist

import joblib
joblib.dump(model, "model/lgbm_model.pkl")
print("Model saved to model/lgbm_model.pkl")


Model saved to model/lgbm_model.pkl


In [ ]:
import joblib

joblib.dump(model, "model/lgbm_model.pkl")
print("Model saved to model/lgbm_model.pkl")


Model saved to model/lgbm_model.pkl


In [ ]:
# predict.py
import joblib
import pandas as pd

model = joblib.load("model/lgbm_model.pkl")

# Example input
sample = pd.DataFrame([{
    "vendor_rating": 4.5,
    "deliverydistance": 3.2,
    "preparationtime": 15
}])

prediction = model.predict_proba(sample)[:, 1][0]
print(f"Predicted probability of interaction: {prediction:.4f}")


Predicted probability of interaction: 0.9999


In [ ]:
import pandas as pd
import joblib

# 1. Load sample submission
sample_df = pd.read_csv("SampleSubmission.csv")

# 2. Load trained model
model = joblib.load("model/lgbm_model.pkl")

# 3. Extract features used during training
# Make sure these match your actual model training features
sample_df["CID"] = sample_df["CID X LOC_NUM X VENDOR"].apply(lambda x: x.split(" X ")[0])
sample_df["LOC_NUM"] = sample_df["CID X LOC_NUM X VENDOR"].apply(lambda x: int(x.split(" X ")[1]))
sample_df["VENDOR"] = sample_df["CID X LOC_NUM X VENDOR"].apply(lambda x: int(x.split(" X ")[2]))

# Select same features as in training
features = ["CID", "LOC_NUM", "VENDOR"]

# Encode categorical customer IDs to match training format
from sklearn.preprocessing import LabelEncoder
cid_encoder = LabelEncoder()
sample_df["CID"] = cid_encoder.fit_transform(sample_df["CID"])

X_sub = sample_df[features]

# 4. Predict probabilities
sample_df["target"] = model.predict_proba(X_sub)[:, 1]

# 5. Keep only required columns and save
submission = sample_df[["CID X LOC_NUM X VENDOR", "target"]]
submission.to_csv("submission.csv", index=False)

print("✅ submission.csv generated successfully.")


✅ submission.csv generated successfully.


In [ ]:
import lightgbm as lgb
import joblib

model = lgb.LGBMClassifier()
model.fit(X_train, y_train)
joblib.dump(model, "model/lgbm_model.pkl")


[LightGBM] [Info] Number of positive: 108478, number of negative: 109703
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012632 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 218181, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497193 -> initscore=-0.011229
[LightGBM] [Info] Start training from score -0.011229


['model/lgbm_model.pkl']

In [ ]:
print(model.feature_name_)


NameError: name 'model' is not defined